Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [10]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [11]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [12]:
class State:
    def __init__(self, state: np.ndarray):
        self.state: np.ndarray = state
    
    def __hash__(self):
        return self.state.tobytes().__hash__()
    
    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)
        return self.state == other
    
    def __lt__(self, other):
        return True
    
    def __getitem__(self, key):
        return self.state[key]
    
    def __setitem__(self, key, value):
        self.state[key] = value
        
    def __repr__(self):
        return str(self.state.ravel())
    
    def __str__(self):
        return str(self.state.ravel())
        
    def copy(self):
        return State(self.state.copy())
    
    

In [13]:
def zero_from_state(state: np.ndarray) -> tuple:
    x, y = [int(i[0]) for i in np.where(state == 0)]
    return x, y


def available_actions(state: np.ndarray) -> list['Action']:
    x, y = zero_from_state(state)
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: State, action: 'Action') -> State:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [14]:
# RANDOMIZE_STEPS = 10
# state = np.random.choice(range(PUZZLE_DIM**2), (PUZZLE_DIM, PUZZLE_DIM), replace=False)
# goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
# for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
#     state = do_action(state, choice(available_actions(state)))
#     print(state)

### Breath-First Search (BFS)

In [15]:
# def bfs(state: State, goal: np.ndarray):
#     path = list()
#     queue = list()
#     visited = set()
#     # Append first element to queue
#     queue.append(state)
#     visited.add(state)
    
#     while queue:
#         current_state = queue.pop(0)
#         actions = available_actions(current_state)
        
#         print(current_state)
        
#         # Check for goal reached
#         if np.array_equal(current_state, goal):
#             return True, path
        
#         # Iterate over all possible actions
#         for action in actions:
#             new_state = do_action(current_state, action)
#             # Expand only if next state is not visited
#             if new_state not in visited:
#                 queue.append(new_state)
#                 visited.add(new_state)
#                 path.append(new_state)
    
#     return False, path
            

In [ ]:
from queue import PriorityQueue
import matplotlib.pyplot as plt

def get_pos(state_game: np.ndarray, n: int) -> tuple[int, int]:
    x, y = [int(i[0]) for i in np.where(state_game == n)]
    return x, y

def manhattan_distance(state: State, goal: np.ndarray, n: int) -> int:
	x1, y1 = get_pos(state.state, n)
	x2, y2 = get_pos(goal, n)
	return abs(x1 - x2) + abs(y1 - y2)

def heuristic(state: State, goal: np.ndarray) -> int:
    # return manhattan_distance(state, goal, 0)
    return sum([manhattan_distance(state, goal, n) for n in range(1, PUZZLE_DIM**2)]) / 2

def astar(state: State, goal: np.ndarray) -> float:
    
	open_set = PriorityQueue()
	closed_set = set()
	open_set.put((0, state))
	came_from = {}
	g_score = {state: 0}
	h_score = {state: heuristic(state, goal)}

	while not open_set.empty():
		h, current = open_set.get()

		closed_set.add(current)
		
		# for i in range(PUZZLE_DIM):
		# 	for j in range(PUZZLE_DIM):
		# 		plt.text(j, i, current.state[i, j], ha='center', va='center', color='white')
		# plt.imshow(current.state, cmap='viridis', interpolation='none')
		# plt.colorbar()
		# plt.show() if h == 0 else
		# plt.clf()

		if np.array_equal(current.state, goal):
			path = []
			while current in came_from:
				path.append(current)
				current = came_from[current]
			path.reverse()
			return True, path

		for action in available_actions(current.state):
			neighbor = do_action(current, action)
			neighbor_past = g_score[current] + 1

			if neighbor not in closed_set and (neighbor not in g_score or neighbor_past < g_score[neighbor]):
				came_from[neighbor] = current
				g_score[neighbor] = neighbor_past
				h_score[neighbor] = neighbor_past + heuristic(neighbor, goal)
				open_set.put((h_score[neighbor], neighbor))

	return False, []

In [17]:
solver = astar

In [ ]:
state = State(np.random.choice(range(PUZZLE_DIM**2), (PUZZLE_DIM, PUZZLE_DIM), replace=False))
goal = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

success, path = solver(state, goal)
print(success)
path

False


[]